# Getting started with RAG in Python

This notebook aims to give a basic introduction to Retrieval-Augmented Generation (RAG) with GPT-4 in Python. The intent is to give an extemely transparent (if simple) runthrough of a basic RAG setup in Python. It is not intended to act as a technical reference for any production RAG-based systems. In such cases you should consider using a dedicated vector database (e.g. Qdrant, Chroma, Vespa) and some dedicated LLM tooling such as LangChain.

Additionally, to use this notebook, you'll need an OpenAI API Key and billing set up for your OpenAI account.

## Prerequisites

To get started, make sure you're using Python 3.10 or greater. Install the following packages:

In [1]:
!pip install 'transformers<=4.34.0' openai torch scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.26.2
    Uninstalling huggingface-hub-0.26.2:
      Successfully uninstalled huggingface-hub-0.26.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

Next, add your API key:

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-api-key-here"

## Concepts

The basic RAG pattern involves two key components: the retriever and the generator. The generator is typically a LLM -- in this case it is going to be GPT-4. The retriever can be any external database, but is commonly centered on some form of vector database loaded with _embeddings_. However it is set up, the aim to have the retriever retrieve information in the form of relevant documents or snippets of documents from its data store and to use these to _augment_ the input (prompt) for the Generator in order to allow it to produce better responses. Sounds simple enough, right?


## The data

For this example, you'll use some short snippets about space missions in 2023. These were taken from [Wikipedia](https://en.wikipedia.org/wiki/2023_in_spaceflight) on 29th October 2023 with small modifications. At the time this notebook was written, GPT-4 had access to data up to October 2021. As such all of these events are outside of its parametric memory. Here are the documents:

In [2]:
documents = [
  "On 14 April, ESA launched the Jupiter Icy Moons Explorer (JUICE) spacecraft to explore Jupiter and its large ice-covered moons following an eight-year transit.",
  "ISRO launched its third lunar mission Chandrayaan-3 on 14 July 2023 at 9:05 UTC; it consists of lander, rover and a propulsion module, and successfully landed in the south pole region of the Moon on 23 August 2023.",
  "Russian lunar lander Luna 25 was launched on 10 August 2023, 23:10 UTC, atop a Soyuz-2.1b rocket from the Vostochny Cosmodrome, it was the first Russian attempt to land a spacecraft on the Moon since the Soviet lander Luna 24 in 1974, it crashed on the Moon on 19 August after technical glitches.",
  "JAXA launched SLIM (Smart Lander for Investigating Moon) lunar lander (carrying a mini rover) and a space telescope (XRISM) on 6 September.",
  "The OSIRIS-REx mission returned to Earth on 24 September with samples collected from asteroid Bennu.",
  "NASA launched the Psyche spacecraft on 13 October 2023, an orbiter mission that will explore the origin of planetary cores by studying the metallic asteroid 16 Psyche, on a Falcon Heavy launch vehicle."
]

## The retriever

In situations where you intend to use an off-the-shelf LLM (i.e. Generator), the retriever is the aspect of the system you have most control over. A common design for retrievers is to use a pre-trained language model to convert your documents into embeddings, and to then use a vector database to store these and query these embeddings during operation.

In this simple example, you'll use a pre-trained embedding model available through the `transformers` library from [Hugging Face](huggingface.co). This [pre-trained model](https://huggingface.co/BAAI/bge-base-en) is the English language version of the 'General Embedding' model from Beijing Academy of Artificial Intelligence (BAAI).

Here's a function that creates embeddings from a set of documents:

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel


def embed_documents(docs, model_name):
  """Embed the provided documents to create a document index"""
  # load the tokenizer and model
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModel.from_pretrained(model_name)

  # encode the docs with the tokenizer
  encoded_docs = tokenizer(
      docs, padding=True, truncation=True,
      return_tensors='pt'
  )

  # generate your output embedding vectors
  with torch.no_grad():
      model_output = model(**encoded_docs)
      doc_embeddings = model_output[0][:, 0]

  # convert to numpy vectors for ease of use
  return doc_embeddings.numpy()

As you can see, there are two main elements here, the `tokenizer` and the `model`. You'll notice both use the same `model_name`. Language models often have their own tokenizers. This allows them to convert from human-readable natural language into machine-readable format compatible with the model. This machine-readable format is then used by the model to generate your embeddings.

You can now generate your document index as:

In [7]:
!pip install --upgrade transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.6/447.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.0
    Uninstalling transformers-4.34.0:
      Successfully uninstalled transformers-4.34.0


In [6]:
document_index = embed_documents(documents, model_name="HuggingFaceH4/zephyr-7b-beta")
document_index[0].shape # shape of each vector in the index

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


RuntimeError: Failed to import transformers.models.mistral.modeling_mistral because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'split_torch_state_dict_into_shards' from 'huggingface_hub' (/usr/local/lib/python3.10/dist-packages/huggingface_hub/__init__.py)

Note that it can be a good idea to 'chunk' your documents before embedding them and creating your document index. This can help reduce the context length subsequently required for the LLM you use, which in turn can improve inference speed and reduce cost. Additionally, it can help the resultant prompt reference specific facts or segments within a document more easily too, which can improve the quality of responses in some cases.

Now that you have your document index, you can create a simple retrieval function to search the index for matching documents:

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def retrieve_documents(query_string, doc_index, docs, k=5, doc_model_name="BAAI/bge-base-en"):
  # embed the query string to obtain a query vector
  query_vector = embed_documents(
      [query_string],
      model_name="BAAI/bge-base-en"
  ).reshape(1, -1)

  # use the query vector to find the most similar document to the query
  similarity = cosine_similarity(query_vector, doc_index).flatten()

  # return the top k most similar docs
  # here, argsort assigns the indices that would order the similarities
  # from least similar to most similar. The [::-1] slice reverses this
  # to return most similar to least, and slices the top k of these
  return [docs[i] for i in np.argsort(similarity)[::-1][:k]]

In this case you're using [cosine similarity](https://developers.google.com/machine-learning/clustering/similarity/measuring-similarity), a common method for comparing embedding vectors. The approach implemented here finds the similarity of your query vector to _every other document vector_. When you have lots of documents, this can be extremely expensive. It is in these circumstances that tools that provide _approximate_ search over the document index are useful. You can check out [FAISS](https://faiss.ai/index.html) from Facebook AI Research as an example of a tool that supports efficient similarity search over very large document indexes. For production environments, this is also where vector databases like [Qdrant](https://qdrant.tech/), [Chroma](https://www.trychroma.com/) or [Vespa](https://vespa.ai/) start to come in handy: they manage efficient similarity search for you!

With all that said, you can then test this simple Retriever (i.e. `retrieve_documents`) using:

In [ ]:
example_retrieved_docs = retrieve_documents(
    "Tell me about the Japanese lunar mission.",
    document_index,
    documents,
    k=3
)

example_retrieved_docs

['JAXA launched SLIM (Smart Lander for Investigating Moon) lunar lander (carrying a mini rover) and a space telescope (XRISM) on 6 September.',
 'ISRO launched its third lunar mission Chandrayaan-3 on 14 July 2023 at 9:05 UTC; it consists of lander, rover and a propulsion module, and successfully landed in the south pole region of the Moon on 23 August 2023.',
 'Russian lunar lander Luna 25 was launched on 10 August 2023, 23:10 UTC, atop a Soyuz-2.1b rocket from the Vostochny Cosmodrome, it was the first Russian attempt to land a spacecraft on the Moon since the Soviet lander Luna 24 in 1974, it crashed on the Moon on 19 August after technical glitches.']

Clearly there are not many documents in this document index. However, you should see that the top document is indeed most relevant to the query text: exactly what you want to see!

With this done, it is time to use the retrieved documents to create an augmented input for the LLM (i.e. create an augmented prompt). You'll use a very simple prompt in this case (you should think about ways to make it better!). Here's a simple function to achieve this:

In [ ]:
def create_augmented_prompt(query_string, docs):
  # concatenate the retrieved docs as context for the LLM
  # you could do other pre-processing here too
  context = "\n".join(docs)
  # define your prompt template
  prompt_template = """Here is some relevant information:
  {context}

  Q: {query}
  A:
  """
  # render the prompt template
  return prompt_template.format(context=context, query=query_string)

And you can see how this behaves with the following:

In [ ]:
example_augmented_prompt = create_augmented_prompt(
    "Tell me about the Japanese lunar mission.",
    example_retrieved_docs
)
example_augmented_prompt

'Here is some relevant information:\n  JAXA launched SLIM (Smart Lander for Investigating Moon) lunar lander (carrying a mini rover) and a space telescope (XRISM) on 6 September.\nISRO launched its third lunar mission Chandrayaan-3 on 14 July 2023 at 9:05 UTC; it consists of lander, rover and a propulsion module, and successfully landed in the south pole region of the Moon on 23 August 2023.\nRussian lunar lander Luna 25 was launched on 10 August 2023, 23:10 UTC, atop a Soyuz-2.1b rocket from the Vostochny Cosmodrome, it was the first Russian attempt to land a spacecraft on the Moon since the Soviet lander Luna 24 in 1974, it crashed on the Moon on 19 August after technical glitches.\n\n  Q: Tell me about the Japanese lunar mission.\n  A:\n  '

The last important piece is querying the LLM itself. This is simple enough. Here is another simple function to query the OpenAI GPT-4 API:

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="sk-10756b0e11834102825b28fd79ba6680", base_url="https://api.deepseek.com")

# Query using DeepSeek
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content":query_string},

    ],
    stream=False
)

print(response.choices[0].message.content)

The capital of France is Paris.


In [ ]:
import openai

def generate_response(query_string, model_name):
  response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content":query_string},

    ],
    stream=False
)
  return response.choices[0].message.content

Once again, you can see how this behaves with:

In [ ]:
generate_response("Hello, world!", model_name="gpt-4")

'Hello! How can I assist you today?'

Okay, you've now seen all of the core components of creating using RAG to query an LLM. Time to bring it all together! 🚀

In [ ]:
def generate_rag_response(
    query_string,
    docs,
    doc_index,
    model_name="gpt-3.5-turbo",
    k=3
):

  # R: retrieve documents
  retrieved_docs = retrieve_documents(
      query_string, doc_index, documents
  )
  # A: create augmented prompt
  augmented_prompt = create_augmented_prompt(query_string, retrieved_docs)

  # G: generate response!
  generated_response = generate_response(augmented_prompt, model_name)
  return generated_response

Now generate a RAG response with:

In [ ]:
generate_rag_response("Tell me about the status of the latest Indian lunar mission.", documents, document_index, model_name="gpt-4")

'The latest Indian lunar mission, Chandrayaan-3, was launched by ISRO on 14 July 2023. It consists of a lander, rover, and a propulsion module. The mission successfully landed in the south pole region of the Moon on 23 August 2023.'

Up-to-date and accurate. Nice. Let's compare the RAG response to a 'raw' GPT-4 response:

In [ ]:
generate_response("Tell me about the status of the latest Indian lunar mission.", model_name="gpt-4")

'As of March 2022, the latest Indian lunar mission was Chandrayaan-3. The mission is being led by the Indian Space Research Organisation (ISRO) and is planned for launch in 2022. After a previous attempt failed to make a soft landing on the moon with Chandrayaan-2 in September 2019, ISRO has been preparing to retry the mission without an orbiter. The mission will include a lander and a rover to study the lunar surface close to the south pole, a largely unexplored area. However, for detailed real-time status, it would be best to refer to official ISRO channels or updates.'

This response is clearly relying on out-of-date information!

## Next steps

This is clearly a very simple example. A production system inevitably comes with a bit more complexity. For one thing, using a good vector database will likely help make your RAG setup faster, more scalable and more flexible. Here are some pointers on what to consider when taking the next step:

* **Prompt engineering** - The prompt in this notebook is about as basic as it gets. Careful prompt engineering can make it possible for the LLM to directly reference specific retrieved documents, and to add hyperlinks and other interactivity into the response. You can also get the LLM to respond in structured formats like JSON, too. The [Prompting Guide](https://www.promptingguide.ai/techniques/rag) is a great resource to help get started with this.
* **Multi-index retrieval** - In this case, you used a single document index. It is not uncommon to need to use multiple document data stores. Many vector databases make it easy to achieve this.
* **Fine-tuning RAG models** - The [original RAG paper](https://arxiv.org/pdf/2005.11401.pdf) discusses how to train an embedding model (i.e. the model used for retrieval task) alongside the generation model (i.e. the language model) to perform better at certain tasks. OpenAI themselves provide a 'cookbook' notebook outlining how you can [fine tune an LLM to produce better RAG responses](https://cookbook.openai.com/examples/fine-tuned_qa/ft_retrieval_augmented_generation_qdrant).
* **Caching** - Thoughtful use of caching layers around the retriever (and possibly the generator) can improve the latency and cost efficiency of your RAG system.

If you're also a bit unfamiliar with embeddings, a great use of your time would be to learn more about how they fit into the world of modern machine learning. There's [a good introduction in the Google ML Developers course](https://developers.google.com/machine-learning/crash-course/embeddings/video-lecture). If you're a visual learner, you may also want to check out the interactive [Embedding Projector](https://projector.tensorflow.org/) to visualise embeddings, too.